Họ tên: Lê Minh Quân

MSSV: 22120291

# HW0: Làm quen với CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

## Câu 1

In [3]:
%%writefile HW0_P1.cu
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    printf("Number of CUDA devices: %d\n\n", deviceCount);

    for (int dev = 0; dev < deviceCount; ++dev) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, dev);

        printf("=== Device %d ===\n", dev);
        printf("GPU name: %s\n", prop.name);
        printf("Compute capability: %d.%d\n", prop.major, prop.minor);
        printf("Total global memory: %zu bytes\n", prop.totalGlobalMem);
        printf("Shared memory per block: %zu bytes\n", prop.sharedMemPerBlock);
        printf("Constant memory: %zu bytes\n", prop.totalConstMem);
        printf("Max threads per block: %d\n", prop.maxThreadsPerBlock);
        printf("Max block dimensions: %d %d %d\n", prop.maxThreadsDim[0], prop.maxThreadsDim[1], prop.maxThreadsDim[2]);
        printf("Max grid dimensions: %d %d %d\n", prop.maxGridSize[0], prop.maxGridSize[1], prop.maxGridSize[2]);
        printf("Warp size: %d\n", prop.warpSize);
        printf("===================\n\n");
    }
    return 0;
}

Writing HW0_P1.cu


In [7]:
!nvcc HW0_P1.cu -o HW0_P1
!./HW0_P1

Number of CUDA devices: 1

=== Device 0 ===
GPU name: Tesla T4
Compute capability: 7.5
Total global memory: 15828320256 bytes
Shared memory per block: 49152 bytes
Constant memory: 65536 bytes
Max threads per block: 1024
Max block dimensions: 1024 1024 64
Max grid dimensions: 2147483647 65535 65535
Warp size: 32



## Câu 2

In [8]:
%%writefile HW0_P2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <chrono>
using namespace std;

#define BLOCK_SIZE 256

// ---------------- VERSION 1 ----------------
// mỗi thread cộng 2 phần tử cách nhau blockDim.x
__global__ void vecAdd_v1(const float *A, const float *B, float *C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idx2 = idx + blockDim.x;
    if (idx < N)
        C[idx] = A[idx] + B[idx];
    if (idx2 < N)
        C[idx2] = A[idx2] + B[idx2];
}

// ---------------- VERSION 2 ----------------
// mỗi thread cộng 2 phần tử liên tiếp
__global__ void vecAdd_v2(const float *A, const float *B, float *C, int N) {
    int idx = blockIdx.x * blockDim.x * 2 + threadIdx.x * 2;
    if (idx < N)
        C[idx] = A[idx] + B[idx];
    if (idx + 1 < N)
        C[idx + 1] = A[idx + 1] + B[idx + 1];
}

// ---------------- HOST ADD ----------------
void vecAdd_host(const float *A, const float *B, float *C, int N) {
    for (int i = 0; i < N; ++i)
        C[i] = A[i] + B[i];
}

// ---------------- MAIN ----------------
int main() {
    int sizes[] = {64, 256, 1024, 4096, 16384, 65536, 262144, 1048576, 4194304, 16777216};
    int num_sizes = sizeof(sizes) / sizeof(int);

    printf("%-12s %-12s %-18s %-18s\n", "VectorSize", "HostTime(ms)", "Device_v1(ms)", "Device_v2(ms)");

    for (int s = 0; s < num_sizes; ++s) {
        int N = sizes[s];
        size_t bytes = N * sizeof(float);

        float *h_A = (float*)malloc(bytes);
        float *h_B = (float*)malloc(bytes);
        float *h_C = (float*)malloc(bytes);
        float *d_A, *d_B, *d_C;
        cudaMalloc(&d_A, bytes);
        cudaMalloc(&d_B, bytes);
        cudaMalloc(&d_C, bytes);

        for (int i = 0; i < N; ++i) {
            h_A[i] = i * 0.5f;
            h_B[i] = i * 0.25f;
        }

        // ---- Host time ----
        auto t1 = chrono::high_resolution_clock::now();
        vecAdd_host(h_A, h_B, h_C, N);
        auto t2 = chrono::high_resolution_clock::now();
        double hostTime = chrono::duration<double, milli>(t2 - t1).count();

        cudaMemcpy(d_A, h_A, bytes, cudaMemcpyHostToDevice);
        cudaMemcpy(d_B, h_B, bytes, cudaMemcpyHostToDevice);

        int grid_v1 = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
        int grid_v2 = (N + BLOCK_SIZE * 2 - 1) / (BLOCK_SIZE * 2);

        cudaEvent_t start, stop;
        float time_v1, time_v2;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        // ---- Version 1 ----
        cudaEventRecord(start);
        vecAdd_v1<<<grid_v1, BLOCK_SIZE>>>(d_A, d_B, d_C, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&time_v1, start, stop);

        // ---- Version 2 ----
        cudaEventRecord(start);
        vecAdd_v2<<<grid_v2, BLOCK_SIZE>>>(d_A, d_B, d_C, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&time_v2, start, stop);

        printf("%-12d %-12.3f %-18.3f %-18.3f\n", N, hostTime, time_v1, time_v2);

        cudaFree(d_A);
        cudaFree(d_B);
        cudaFree(d_C);
        free(h_A);
        free(h_B);
        free(h_C);
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    return 0;
}

Writing HW0_P2.cu


In [9]:
!nvcc HW0_P2.cu -o HW0_P2
!./HW0_P2

VectorSize   HostTime(ms) Device_v1(ms)      Device_v2(ms)     
64           0.001        43.921             0.003             
256          0.002        0.003              0.002             
1024         0.005        0.003              0.002             
4096         0.018        0.002              0.002             
16384        0.066        0.002              0.002             
65536        0.251        0.002              0.002             
262144       1.537        0.002              0.002             
1048576      5.983        0.002              0.002             
4194304      24.230       0.003              0.002             
16777216     99.628       0.003              0.002             
